In [1]:
import cv2
import numpy as np
#from matplotlib import pyplot as plt

In [ ]:
def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img

In [2]:
#img = cv2.imread("vi_osmo_numbers/color/Photo 2-14-17, 10 44 20 AM.jpg", cv2.IMREAD_COLOR)
img = cv2.imread("vi_osmo_numbers/cropped_number_rotated/o2.jpg_cropped.png")

In [3]:
#plt.imshow(img)
#plt.show()
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#cv2.imshow('imgray',imgray)

edge_wide = cv2.Canny(imgray, 10, 200)
edge_tight = cv2.Canny(imgray, 225, 250)
edge_custom = cv2.Canny(imgray, 200, 400)

sigma=0.33
# compute the median of the single channel pixel intensities
v = np.median(imgray)

# apply automatic Canny edge detection using the computed median
lower = int(max(0, (1.0 - sigma) * v))
upper = int(min(255, (1.0 + sigma) * v))
edge_auto = cv2.Canny(imgray, lower, upper)

cv2.imshow('edge_wide', edge_wide)
cv2.waitKey(0)
cv2.imshow('edge_tight', edge_tight)
cv2.waitKey(0)
cv2.imshow('edge_custom', edge_custom)
cv2.waitKey(0)
cv2.imshow('edge_auto', edge_auto)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
img_blurred = cv2.medianBlur(imgray, 5)

ret,th1 = cv2.threshold(img_blurred,127,255,cv2.THRESH_BINARY_INV)
th2 = cv2.adaptiveThreshold(img_blurred,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY_INV,11,2)
th3 = cv2.adaptiveThreshold(img_blurred,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY_INV,11,2)

otsu_threshold1, th4 = cv2.threshold(imgray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

#otsu_threshold2, th5 = cv2.threshold(img_blurred, 0, 255, cv2.THRESH_BINARYINV+cv2.THRESH_OTSU)

#cv2.imshow("Global Thresholding (v = 127)", th1)
#cv2.waitKey(0)
#cv2.imshow("Adaptive Mean Thresholding", th2)
#cv2.waitKey(0)
#cv2.imshow("Adaptive Gaussian Thresholding", th3)
#cv2.waitKey(0)

cv2.imshow("Otsu th4", th4)
cv2.waitKey(0)

#cv2.imshow("Otsu th5", th4)
#cv2.waitKey(0)

cv2.destroyAllWindows()

In [ ]:
cimg, contours, hierarchy = cv2.findContours(th2, 1, 2)
cnt = contours[0]

In [ ]:
cv2.imshow("contours", cnt)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

def run_main():
#     cap = cv2.VideoCapture(0)
#     cap.set(cv2.cv.CV_CAP_PROP_FRAME_WIDTH, 1280)
#     cap.set(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT, 720)

#     while(True):
    #ret, frame = cap.read()
    #roi = frame[0:500, 0:500]
    roi = cv2.imread("vi_osmo_numbers/o2.jpg")
    #roi = cv2.imread("coin_img.png")
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    gray_blur = cv2.GaussianBlur(gray, (15, 15), 0)
    thresh = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 11, 1)

    kernel = np.ones((3, 3), np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE,
        kernel, iterations=4)

    closing = cv2.bitwise_not(closing)
    cont_img = closing.copy()
    cimg, contours, hierarchy = cv2.findContours(cont_img, cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        #print area
        #perimeter = cv2.arcLength(cnt, True)
        #print perimeter
#         x,y,w,h = cv2.boundingRect(cnt)
#         cv2.rectangle(roi,(x,y),(x+w,y+h),(0,255,0),2)
#         cv2.waitKey(0)
        
#         if area < 2000 or area > 4000:
#         #if area > 100:
#              continue

        if len(cnt) < 5:
            continue

        #ellipse = cv2.fitEllipse(cnt)
        #cv2.ellipse(roi, ellipse, (0,255,0), 2)
        x,y,w,h = cv2.boundingRect(cnt)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

    cv2.imshow("Morphological Closing", closing)
    cv2.imshow("Adaptive Thresholding", thresh)
    cv2.imshow('Contours', roi)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

#     cap.release()

#     x,y,w,h = cv2.boundingRect(contours[0])
#     cv2.rectangle(roi,(x,y),(x+w,y+h),(0,255,0),2)
#     cv2.waitKey(0)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    run_main()

In [ ]:
gray.shape[1]

In [ ]:
import cv2
import numpy as np
import math
import numpy.linalg as la


SZ = 20
affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR
def deskew(img):
    size = (img.shape[1], img.shape[0])
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img, M, size, flags=affine_flags)
    return img

for n in xrange(1, 10):
    filename = "o" + str(n) + ".jpg"
    img = cv2.imread("vi_osmo_numbers/color/" + filename)
    #roi = cv2.imread("coin_img.png")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # filter for edge
    #gray_blur = cv2.bilateralFilter(gray, 11, 17, 17)
    gray_blur = cv2.bilateralFilter(gray, 11, 13, 13)
    #gray_blur = gray

    sigma=0.33
    # compute the median of the single channel pixel intensities
    v = np.median(gray_blur)
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(gray_blur, lower, upper)
    #otsu_threshold, edged = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #edged = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 1)
    # cv2.imshow("edged", edged)
    # cv2.waitKey(0)
    #cv2.destroyAllWindows()

    # find contours in the edged image, keep only the largest
    # ones, and initialize our screen contour
    (cimg, cnts, hierarchy) = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #(cimg, cnts, hierarchy) = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    #(cimg, cnts, hierarchy) = cv2.findContours(edged.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    #cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
    cnts_idxes = sorted(range(len(cnts)), key=lambda k: cv2.contourArea(cnts[k]), 
                        reverse = True)[:10]
    #hierarchy = sorted(hierarchy, key = cv2.contourArea, reverse = True)[:5]
    screenCnt = None

    #print "length of hierarchy[0]", len(hierarchy[0])
    #print "length of cnts_idxes", len(cnts_idxes)

    bFoundTarget = False
    # loop over our contour
    #for c, hi in zip(cnts, hierarchy[0]):
    for ci in cnts_idxes:
        c = cnts[ci]
        hi = hierarchy[0][ci]

    #     # display contours and its area
    #     mask = np.zeros(gray.shape,np.uint8)
    #     cv2.drawContours(mask,[c],0,255,-1)
    #     cv2.imshow(str(ci)+"mask", mask)

        #print cv2.isContourConvex(c)
        #print hi
        #if hi[0] == -1 and hi[1] == -1 and hi[2] == -1 and hi[3] != -1:
        #if hi[0] == -1 and hi[1] == -1 and hi[2] != -1 and hi[3] != -1:
        if hi[3] != -1:
    #         # display contours and its area
    #         mask = np.zeros(gray.shape,np.uint8)
    #         cv2.drawContours(mask,[c],0,255,-1)
    #         cv2.imshow(str(ci)+"mask", mask)

    #         # draw contours on original image
    #         img_copy = img.copy()
    #         cv2.drawContours(img_copy, [c], -1, (0, 255, 0), 3)
    #         cv2.imshow(str(ci)+"contours", img_copy)
    #         #cv2.waitKey(0)

            # approximate the contour
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)

            if len(approx) == 4:
                continue

    #         # crop target part
    #         x,y,w,h = cv2.boundingRect(c)
    #         cropped = img.copy()[y:y+h,x:x+w]
    #         #cropped_gray = gray[y:y+h,x:x+w]
    #         #cropped = img.copy()[y+5:y+h-5,x+5:x+w-5]
    #         #cropped_gray = gray[y+5:y+h-5,x+5:x+w-5]
    #         #cv2.imshow("cropped", cropped)

            # check parent part
            parent_idx = hi[3] 
            while parent_idx != -1:
                #print "parent_idx", parent_idx
                # parent contour
                parent_c = cnts[parent_idx]
                parent_hi = hierarchy[0][parent_idx]
                # approximate the contour
                peri = cv2.arcLength(parent_c, True)
                approx = cv2.approxPolyDP(parent_c, 0.11 * peri, True)        
                # if our approximated contour has four points, then
                # we can assume that we have found our screen
                #print "parent approx: ", len(approx)
                x,y,w,h = cv2.boundingRect(parent_c)
                aspect_ratio = w / h
                #print "aspect_ratio of ", parent_idx, aspect_ratio
                if aspect_ratio >=1 and len(approx) == 4:
                    #print "aspect_ratio of ", parent_idx, aspect_ratio
                    parent_cropped = img.copy()[y:y+h,x:x+w]
                    #cropped_gray = gray[y:y+h,x:x+w]
                    #cropped = img.copy()[y+5:y+h-5,x+5:x+w-5]
                    #cropped_gray = gray[y+5:y+h-5,x+5:x+w-5]                

#                     # for child - target number
#                     child_idx = parent_hi[2]
#                     child_c = cnts[child_idx]
#                     # draw contours on original image
#                     img_copy = img.copy()
#                     cv2.drawContours(img_copy, [child_c], -1, (0, 255, 0), 3)
#                     cv2.imshow(str(child_idx)+"contours", img_copy)
#                     cv2.imshow("p"+str(child_idx), parent_cropped)

                    bFoundTarget = True
                    break
                else:
                    parent_idx = parent_hi[3]

            #
            if bFoundTarget:
                break



    #     # display contours and its area
    #     mask = np.zeros(gray.shape,np.uint8)
    #     cv2.drawContours(mask,[c],0,255,-1)
    #     cv2.imshow(str(hi), mask)
    #     cv2.waitKey(0)

    #     x,y,w,h = cv2.boundingRect(c)
    #     aspect_ratio = round(w / h);
    #     #print aspect_ratio

    #     #f h[3] != -1:
    #     if aspect_ratio == 1:
    #         #print "aaaa", h        
    #         #,y,w,h = cv2.boundingRect(c)
    #         #spect_ratio = round(w / h);
    #         #print aspect_ratio
    # #         if aspect_ratio == 1:
    #         mask = img.copy()[y:y+h,x:x+w]
    #         mask_gray = gray[y:y+h,x:x+w]
    #         mask_deskewed = deskew(mask_gray)
    #         #cv2.imshow("shape", mask)
    #         #cv2.waitKey(0)
    #         #break



    #     # approximate the contour
    #     peri = cv2.arcLength(c, True)
    #     #approx = cv2.approxPolyDP(c, 0.02 * peri, True)    
    #     approx = cv2.approxPolyDP(c, 0.002 * peri, True)


        # if our approximated contour has four points, then
        # we can assume that we have found our screen
    #     if len(approx) == 4:
    #         screenCnt = approx
    #         #print screenCnt
    #         #print IsSquare(approx)

    #         x,y,w,h = cv2.boundingRect(c)
    #         cropped = img.copy()[y:y+h,x:x+w]
            #cropped_gray = gray[y:y+h,x:x+w]
            #cropped = img.copy()[y+5:y+h-5,x+5:x+w-5]
            #cropped_gray = gray[y+5:y+h-5,x+5:x+w-5]

    #         cropped_edged = edged[y+5:y+h-5,x+5:x+w-5]        
    #         (_, cns, __) = cv2.findContours(cropped_edged.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    #         cns = sorted(cns, key = cv2.contourArea, reverse = True)[:5]
    #         for cn, j in zip(cns, __[0]):
    #             #print cv2.isContourConvex(cn)            
    #             #print j
    #             # display contours and its area
    #             mask = np.zeros(cropped_edged.shape,np.uint8)
    #             cv2.drawContours(mask,[cn],0,255,-1)
    #             cv2.imshow(str(j), mask)
    #             cv2.waitKey(0)

            #cropped_deskewed = deskew(cropped_gray)

    #         cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

    #         rect = cv2.minAreaRect(c)
    #         box = cv2.boxPoints(rect)
    #         box = np.int0(box)
    #         cv2.drawContours(img,[box],0,(0,0,255),2)

    #         #orientation
    #         (x,y),(MA,ma),angle = cv2.fitEllipse(c)
    #         print angle
            #print len(c)
            #pixelpoints = np.transpose(np.nonzero(mask))
            #pixelpoints = cv2.findNonZero(mask)        

            #break

    # Threshold cropped so it becomes binary
    #otsu_threshold, cropped_binary = cv2.threshold(cropped_gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    # kernel = np.ones((3, 3), np.uint8)
    # cropped_binary = cv2.morphologyEx(cropped_binary, cv2.MORPH_CLOSE, kernel, iterations=2)
    # #closing = cv2.bitwise_not(closing)

    #cv2.imshow("edged", edged)
    #cv2.drawContours(img, [screenCnt], -1, (0, 255, 0), 3)
    #cv2.drawContours(img, [box], -1, (0, 255, 0), 3)
    #cv2.imshow("digit tag", img_copy)
    #cv2.imshow("cropped", cropped)
    #cv2.imshow("gray", mask_gray)
    #cv2.waitKey(0)
    #cv2.destroyWindow("gray")
    #cv2.imshow("deskewed", mask_deskewed)
    #cv2.imshow("cropped_edged", cropped_edged)

    #cv2.waitKey(0)
    cv2.destroyAllWindows()

    #horizontal_cropped = cv2.flip( mask, 1 )
    #cv2.imwrite(filename + "_cropped.png", cropped)
    cv2.imwrite(filename + "_numbox.png", parent_cropped)

In [ ]:
#print otsu_threshold

print len(cnts)


#cv2.imwrite("cropped.png", horizontal_cropped)

In [ ]:
#horizontal_cropped.shape
for s in stats:
    print s[cv2.CC_STAT_AREA]

In [ ]:
# You need to choose 4 or 8 for connectivity type
connectivity = 4  
# Perform the operation
output = cv2.connectedComponentsWithStats(cropped_binary, connectivity, cv2.CV_32S)
# Get the results
# The first cell is the number of labels
num_labels = output[0]
# The second cell is the label matrix
labels = output[1]
# The third cell is the stat matrix
stats = output[2]
# The fourth cell is the centroid matrix
centroids = output[3]

# loop over the unique components
for l in np.unique(labels):
    # if this is the background label, ignore it
    if l == 0:
        continue

    # otherwise, construct the label mask and count the
    # number of pixels 
    labelMask = np.zeros(cropped_binary.shape, dtype="uint8")
    labelMask[labels == l] = 255
    cv2.circle(labelMask, (int(centroids[l][0]), int(centroids[l][1])), 5, (255,0,0), -1)
    #numPixels = cv2.countNonZero(labelMask)
    cv2.imshow(l.tostring(), labelMask)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
centroids[l][0]

In [ ]:
#cropped_dpi = 635, 635

horizontal_cropped = cv2.flip( cropped, 1 )
rgb_im = cv2.cvtColor(horizontal_cropped, cv2.COLOR_BGR2RGB)
pil_im = Image.fromarray(rgb_im)
#pil_im.info["dpi"] = cropped_dpi
#pil_im.info["default"] = [635, 635]
pil_im.info["dpi"] = (300, 300)

#print pil_im.info["dpi"]
#print pil_im.info["default"]

#pil_im.save("pil_cropped.png")

pil_im.save("pil_cropped.png", dpi=(300, 300))

#im2 = Image.open("pil_cropped.png")
#im2.info

#pil_im = pil_im.resize((635, 635), Image.ANTIALIAS)

In [ ]:
import pytesseract
import Image

#pil_im = Image.fromarray(horizontal_cropped)
#pil_im = Image.fromarray(np.roll(horizontal_cropped, 1, axis=-1))



recognizedTxt = pytesseract.image_to_string(pil_im, lang='eng', 
                                  config="-psm 10")
#pil_im.show()

import string
all = string.maketrans('','')
nodigs = all.translate(all, string.digits)
print recognizedTxt.translate(all, nodigs)